In [1]:
# tokenization
# some issues with it and it's complicated but here we go!
# GPT2 paper - byte-level encoding
# tokenizer with vocabulary of 50257 tokens
# context size is 1024 tokens (for gpt2)
# every single token is attending to the previous tokens in the sequence, and they can see up to 1024 tokens
# process of treating sequence of text into tokens
# For LLama2 paper - 2 TRILLION tokens of data  -> that's their performance-to-cost trade-off
# byte pair encoding algo is what we're going to implement here!

In [ ]:
# NOTE: tokenization potentially creates a ton of issues with the model (things like spelling tasks & single string processing, non-english processing)
# more tokens used if we're training multiple languages for instance - uses too many tokens & overall context is much shorter
# also based on how the text itself is parsed/parasable
# code with indented whitespace often is token-dense

# Note - tokenizer bytespace can change the overall impact of tokens - allows denser input to transformer
# side-note: is it possible to SUMMARIZE context further out in "history" space to allow for a deeper/longer understanding of context?

In [3]:
# Note: unicode has 151k characters which is much higher than a typical gpt-style tokenizer (but not llama)
# unicode to byte : ord()
ord('h')

104

In [5]:
print([ord(x) for x in "hello world"])

[104, 101, 108, 108, 111, 32, 119, 111, 114, 108, 100]


In [9]:
# you can encode a string natively in python
print(list("hello world".encode("utf-8")))
print(list("hello world".encode("utf-16")))
print(list("hello world".encode("utf-32")))
# Note that utf 16 and utf 32 have a lot of leading zeroes for english text, so it's wasteful as an encoder for most english characters
# what's the right move here?? -> byte-pair encoding algo can give us variable-length encoding 
# even BETTER: getting byte sequences to feed directly into a transformer (only been proposed hypothetically, not done yet in production)
# paper: megabyte: prediction million byte sequences with multiscale transformers

[104, 101, 108, 108, 111, 32, 119, 111, 114, 108, 100]
[255, 254, 104, 0, 101, 0, 108, 0, 108, 0, 111, 0, 32, 0, 119, 0, 111, 0, 114, 0, 108, 0, 100, 0]
[255, 254, 0, 0, 104, 0, 0, 0, 101, 0, 0, 0, 108, 0, 0, 0, 108, 0, 0, 0, 111, 0, 0, 0, 32, 0, 0, 0, 119, 0, 0, 0, 111, 0, 0, 0, 114, 0, 0, 0, 108, 0, 0, 0, 100, 0, 0, 0]


In [ ]:
# byte pair encoding algo -> find pair of tokens that occur the most frequently
# once we find that pair, we replace it with a single token that is appended to our table
# then we continue -> greedy algo style
# ex: aaabdaaabac -> aa -> ZabdZabac -> ab -> ZYdZYac -> ZY -> XdXac -> (no longer have byte pairs that occur more than once)
# X = ZY; Y=ab; Z=aa; XdXac, 

In [1]:
rawtext = """
Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺‌🇳‌🇮‌🇨‌🇴‌🇩‌🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to “support Unicode” in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don’t blame programmers for still finding the whole thing mysterious, even 30 years after Unicode’s inception.

A few months ago, I got interested in Unicode and decided to spend some time learning more about it in detail. In this article, I’ll give an introduction to it from a programmer’s point of view.

I’m going to focus on the character set and what’s involved in working with strings and files of Unicode text. However, in this article I’m not going to talk about fonts, text layout/shaping/rendering, or localization in detail—those are separate issues, beyond my scope (and knowledge) here.

    Diversity and Inherent Complexity
    The Unicode Codespace
        Codespace Allocation
        Scripts
        Usage Frequency
    Encodings
        UTF-8
        UTF-16
    Combining Marks
        Canonical Equivalence
        Normalization Forms
        Grapheme Clusters
    And More…

Diversity and Inherent Complexity

As soon as you start to study Unicode, it becomes clear that it represents a large jump in complexity over character sets like ASCII that you may be more familiar with. It’s not just that Unicode contains a much larger number of characters, although that’s part of it. Unicode also has a great deal of internal structure, features, and special cases, making it much more than what one might expect a mere “character set” to be. We’ll see some of that later in this article.

When confronting all this complexity, especially as an engineer, it’s hard not to find oneself asking, “Why do we need all this? Is this really necessary? Couldn’t it be simplified?”

However, Unicode aims to faithfully represent the entire world’s writing systems. The Unicode Consortium’s stated goal is “enabling people around the world to use computers in any language”. And as you might imagine, the diversity of written languages is immense! To date, Unicode supports 135 different scripts, covering some 1100 languages, and there’s still a long tail of over 100 unsupported scripts, both modern and historical, which people are still working to add.

Given this enormous diversity, it’s inevitable that representing it is a complicated project. Unicode embraces that diversity, and accepts the complexity inherent in its mission to include all human writing systems. It doesn’t make a lot of trade-offs in the name of simplification, and it makes exceptions to its own rules where necessary to further its mission.

Moreover, Unicode is committed not just to supporting texts in any single language, but also to letting multiple languages coexist within one text—which introduces even more complexity.

Most programming languages have libraries available to handle the gory low-level details of text manipulation, but as a programmer, you’ll still need to know about certain Unicode features in order to know when and how to apply them. It may take some time to wrap your head around it all, but don’t be discouraged—think about the billions of people for whom your software will be more accessible through supporting text in their language. Embrace the complexity!
The Unicode Codespace

Let’s start with some general orientation. The basic elements of Unicode—its “characters”, although that term isn’t quite right—are called code points. Code points are identified by number, customarily written in hexadecimal with the prefix “U+”, such as U+0041 “A” latin capital letter a or U+03B8 “θ” greek small letter theta. Each code point also has a short name, and quite a few other properties, specified in the Unicode Character Database.

The set of all possible code points is called the codespace. The Unicode codespace consists of 1,114,112 code points. However, only 128,237 of them—about 12% of the codespace—are actually assigned, to date. There’s plenty of room for growth! Unicode also reserves an additional 137,468 code points as “private use” areas, which have no standardized meaning and are available for individual applications to define for their own purposes.
Codespace Allocation

To get a feel for how the codespace is laid out, it’s helpful to visualize it. Below is a map of the entire codespace, with one pixel per code point. It’s arranged in tiles for visual coherence; each small square is 16×16 = 256 code points, and each large square is a “plane” of 65,536 code points. There are 17 planes altogether.

Map of the Unicode codespace (click to zoom)

White represents unassigned space. Blue is assigned code points, green is private-use areas, and the small red area is surrogates (more about those later). As you can see, the assigned code points are distributed somewhat sparsely, but concentrated in the first three planes.

Plane 0 is also known as the “Basic Multilingual Plane”, or BMP. The BMP contains essentially all the characters needed for modern text in any script, including Latin, Cyrillic, Greek, Han (Chinese), Japanese, Korean, Arabic, Hebrew, Devanagari (Indian), and many more.

(In the past, the codespace was just the BMP and no more—Unicode was originally conceived as a straightforward 16-bit encoding, with only 65,536 code points. It was expanded to its current size in 1996. However, the vast majority of code points in modern text belong to the BMP.)

Plane 1 contains historical scripts, such as Sumerian cuneiform and Egyptian hieroglyphs, as well as emoji and various other symbols. Plane 2 contains a large block of less-common and historical Han characters. The remaining planes are empty, except for a small number of rarely-used formatting characters in Plane 14; planes 15–16 are reserved entirely for private use.
Scripts

Let’s zoom in on the first three planes, since that’s where the action is:

Map of scripts in Unicode planes 0–2 (click to zoom)

This map color-codes the 135 different scripts in Unicode. You can see how Han () and Korean () take up most of the range of the BMP (the left large square). By contrast, all of the European, Middle Eastern, and South Asian scripts fit into the first row of the BMP in this diagram.

Many areas of the codespace are adapted or copied from earlier encodings. For example, the first 128 code points of Unicode are just a copy of ASCII. This has clear benefits for compatibility—it’s easy to losslessly convert texts from smaller encodings into Unicode (and the other direction too, as long as no characters outside the smaller encoding are used).
Usage Frequency

One more interesting way to visualize the codespace is to look at the distribution of usage—in other words, how often each code point is actually used in real-world texts. Below is a heat map of planes 0–2 based on a large sample of text from Wikipedia and Twitter (all languages). Frequency increases from black (never seen) through red and yellow to white.

Heat map of code point usage frequency in Unicode planes 0–2 (click to zoom)

You can see that the vast majority of this text sample lies in the BMP, with only scattered usage of code points from planes 1–2. The biggest exception is emoji, which show up here as the several bright squares in the bottom row of plane 1.
Encodings

We’ve seen that Unicode code points are abstractly identified by their index in the codespace, ranging from U+0000 to U+10FFFF. But how do code points get represented as bytes, in memory or in a file?

The most convenient, computer-friendliest (and programmer-friendliest) thing to do would be to just store the code point index as a 32-bit integer. This works, but it consumes 4 bytes per code point, which is sort of a lot. Using 32-bit ints for Unicode will cost you a bunch of extra storage, memory, and performance in bandwidth-bound scenarios, if you work with a lot of text.

Consequently, there are several more-compact encodings for Unicode. The 32-bit integer encoding is officially called UTF-32 (UTF = “Unicode Transformation Format”), but it’s rarely used for storage. At most, it comes up sometimes as a temporary internal representation, for examining or operating on the code points in a string.

Much more commonly, you’ll see Unicode text encoded as either UTF-8 or UTF-16. These are both variable-length encodings, made up of 8-bit or 16-bit units, respectively. In these schemes, code points with smaller index values take up fewer bytes, which saves a lot of memory for typical texts. The trade-off is that processing UTF-8/16 texts is more programmatically involved, and likely slower.
UTF-8

In UTF-8, each code point is stored using 1 to 4 bytes, based on its index value.

UTF-8 uses a system of binary prefixes, in which the high bits of each byte mark whether it’s a single byte, the beginning of a multi-byte sequence, or a continuation byte; the remaining bits, concatenated, give the code point index. This table shows how it works:
UTF-8 (binary) 	Code point (binary) 	Range
0xxxxxxx 	xxxxxxx 	U+0000–U+007F
110xxxxx 10yyyyyy 	xxxxxyyyyyy 	U+0080–U+07FF
1110xxxx 10yyyyyy 10zzzzzz 	xxxxyyyyyyzzzzzz 	U+0800–U+FFFF
11110xxx 10yyyyyy 10zzzzzz 10wwwwww 	xxxyyyyyyzzzzzzwwwwww 	U+10000–U+10FFFF

A handy property of UTF-8 is that code points below 128 (ASCII characters) are encoded as single bytes, and all non-ASCII code points are encoded using sequences of bytes 128–255. This has a couple of nice consequences. First, any strings or files out there that are already in ASCII can also be interpreted as UTF-8 without any conversion. Second, lots of widely-used string programming idioms—such as null termination, or delimiters (newlines, tabs, commas, slashes, etc.)—will just work on UTF-8 strings. ASCII bytes never occur inside the encoding of non-ASCII code points, so searching byte-wise for a null terminator or a delimiter will do the right thing.

Thanks to this convenience, it’s relatively simple to extend legacy ASCII programs and APIs to handle UTF-8 strings. UTF-8 is very widely used in the Unix/Linux and Web worlds, and many programmers argue UTF-8 should be the default encoding everywhere.

However, UTF-8 isn’t a drop-in replacement for ASCII strings in all respects. For instance, code that iterates over the “characters” in a string will need to decode UTF-8 and iterate over code points (or maybe grapheme clusters—more about those later), not bytes. When you measure the “length” of a string, you’ll need to think about whether you want the length in bytes, the length in code points, the width of the text when rendered, or something else.
UTF-16

The other encoding that you’re likely to encounter is UTF-16. It uses 16-bit words, with each code point stored as either 1 or 2 words.

Like UTF-8, we can express the UTF-16 encoding rules in the form of binary prefixes:
UTF-16 (binary) 	Code point (binary) 	Range
xxxxxxxxxxxxxxxx 	xxxxxxxxxxxxxxxx 	U+0000–U+FFFF
110110xxxxxxxxxx 110111yyyyyyyyyy 	xxxxxxxxxxyyyyyyyyyy + 0x10000 	U+10000–U+10FFFF

A more common way that people talk about UTF-16 encoding, though, is in terms of code points called “surrogates”. All the code points in the range U+D800–U+DFFF—or in other words, the code points that match the binary prefixes 110110 and 110111 in the table above—are reserved specifically for UTF-16 encoding, and don’t represent any valid characters on their own. They’re only meant to occur in the 2-word encoding pattern above, which is called a “surrogate pair”. Surrogate code points are illegal in any other context! They’re not allowed in UTF-8 or UTF-32 at all.

Historically, UTF-16 is a descendant of the original, pre-1996 versions of Unicode, in which there were only 65,536 code points. The original intention was that there would be no different “encodings”; Unicode was supposed to be a straightforward 16-bit character set. Later, the codespace was expanded to make room for a long tail of less-common (but still important) Han characters, which the Unicode designers didn’t originally plan for. Surrogates were then introduced, as—to put it bluntly—a kludge, allowing 16-bit encodings to access the new code points.

Today, Javascript uses UTF-16 as its standard string representation: if you ask for the length of a string, or iterate over it, etc., the result will be in UTF-16 words, with any code points outside the BMP expressed as surrogate pairs. UTF-16 is also used by the Microsoft Win32 APIs; though Win32 supports either 8-bit or 16-bit strings, the 8-bit version unaccountably still doesn’t support UTF-8—only legacy code-page encodings, like ANSI. This leaves UTF-16 as the only way to get proper Unicode support in Windows. (Update: in Win10 version 1903, they finally added UTF-8 support to the 8-bit APIs! 😊)

By the way, UTF-16’s words can be stored either little-endian or big-endian. Unicode has no opinion on that issue, though it does encourage the convention of putting U+FEFF zero width no-break space at the top of a UTF-16 file as a byte-order mark, to disambiguate the endianness. (If the file doesn’t match the system’s endianness, the BOM will be decoded as U+FFFE, which isn’t a valid code point.)
Combining Marks

In the story so far, we’ve been focusing on code points. But in Unicode, a “character” can be more complicated than just an individual code point!

Unicode includes a system for dynamically composing characters, by combining multiple code points together. This is used in various ways to gain flexibility without causing a huge combinatorial explosion in the number of code points.

In European languages, for example, this shows up in the application of diacritics to letters. Unicode supports a wide range of diacritics, including acute and grave accents, umlauts, cedillas, and many more. All these diacritics can be applied to any letter of any alphabet—and in fact, multiple diacritics can be used on a single letter.

If Unicode tried to assign a distinct code point to every possible combination of letter and diacritics, things would rapidly get out of hand. Instead, the dynamic composition system enables you to construct the character you want, by starting with a base code point (the letter) and appending additional code points, called “combining marks”, to specify the diacritics. When a text renderer sees a sequence like this in a string, it automatically stacks the diacritics over or under the base letter to create a composed character.

For example, the accented character “Á” can be expressed as a string of two code points: U+0041 “A” latin capital letter a plus U+0301 “◌́” combining acute accent. This string automatically gets rendered as a single character: “Á”.

Now, Unicode does also include many “precomposed” code points, each representing a letter with some combination of diacritics already applied, such as U+00C1 “Á” latin capital letter a with acute or U+1EC7 “ệ” latin small letter e with circumflex and dot below. I suspect these are mostly inherited from older encodings that were assimilated into Unicode, and kept around for compatibility. In practice, there are precomposed code points for most of the common letter-with-diacritic combinations in European-script languages, so they don’t use dynamic composition that much in typical text.

Still, the system of combining marks does allow for an arbitrary number of diacritics to be stacked on any base character. The reductio-ad-absurdum of this is Zalgo text, which works by ͖͟ͅr͞aṋ̫̠̖͈̗d͖̻̹óm̪͙͕̗̝ļ͇̰͓̳̫ý͓̥̟͍ ̕s̫t̫̱͕̗̰̼̘͜a̼̩͖͇̠͈̣͝c̙͍k̖̱̹͍͘i̢n̨̺̝͇͇̟͙ģ̫̮͎̻̟ͅ ̕n̼̺͈͞u̮͙m̺̭̟̗͞e̞͓̰̤͓̫r̵o̖ṷs҉̪͍̭̬̝̤ ̮͉̝̞̗̟͠d̴̟̜̱͕͚i͇̫̼̯̭̜͡ḁ͙̻̼c̲̲̹r̨̠̹̣̰̦i̱t̤̻̤͍͙̘̕i̵̜̭̤̱͎c̵s ͘o̱̲͈̙͖͇̲͢n͘ ̜͈e̬̲̠̩ac͕̺̠͉h̷̪ ̺̣͖̱ḻ̫̬̝̹ḙ̙̺͙̭͓̲t̞̞͇̲͉͍t̷͔̪͉̲̻̠͙e̦̻͈͉͇r͇̭̭̬͖,̖́ ̜͙͓̣̭s̘̘͈o̱̰̤̲ͅ ̛̬̜̙t̼̦͕̱̹͕̥h̳̲͈͝ͅa̦t̻̲ ̻̟̭̦̖t̛̰̩h̠͕̳̝̫͕e͈̤̘͖̞͘y҉̝͙ ̷͉͔̰̠o̞̰v͈͈̳̘͜er̶f̰͈͔ḻ͕̘̫̺̲o̲̭͙͠ͅw̱̳̺ ͜t̸h͇̭͕̳͍e̖̯̟̠ ͍̞̜͔̩̪͜ļ͎̪̲͚i̝̲̹̙̩̹n̨̦̩̖ḙ̼̲̼͢ͅ ̬͝s̼͚̘̞͝p͙̘̻a̙c҉͉̜̤͈̯̖i̥͡n̦̠̱͟g̸̗̻̦̭̮̟ͅ ̳̪̠͖̳̯̕a̫͜n͝d͡ ̣̦̙ͅc̪̗r̴͙̮̦̹̳e͇͚̞͔̹̫͟a̙̺̙ț͔͎̘̹ͅe̥̩͍ a͖̪̜̮͙̹n̢͉̝ ͇͉͓̦̼́a̳͖̪̤̱p̖͔͔̟͇͎͠p̱͍̺ę̲͎͈̰̲̤̫a̯͜r̨̮̫̣̘a̩̯͖n̹̦̰͎̣̞̞c̨̦̱͔͎͍͖e̬͓͘ ̤̰̩͙̤̬͙o̵̼̻̬̻͇̮̪f̴ ̡̙̭͓͖̪̤“̸͙̠̼c̳̗͜o͏̼͙͔̮r̞̫̺̞̥̬ru̺̻̯͉̭̻̯p̰̥͓̣̫̙̤͢t̳͍̳̖ͅi̶͈̝͙̼̙̹o̡͔n̙̺̹̖̩͝ͅ”̨̗͖͚̩.̯͓

A few other places where dynamic character composition shows up in Unicode:

    Vowel-pointing notation in Arabic and Hebrew. In these languages, words are normally spelled with some of their vowels left out. They then have diacritic notation to indicate the vowels (used in dictionaries, language-teaching materials, children’s books, and such). These diacritics are expressed with combining marks.
    A Hebrew example, with niqqud:	אֶת דַלְתִּי הֵזִיז הֵנִיעַ, קֶטֶב לִשְׁכַּתִּי יָשׁוֹד
    Normal writing (no niqqud):	את דלתי הזיז הניע, קטב לשכתי ישוד

    Devanagari, the script used to write Hindi, Sanskrit, and many other South Asian languages, expresses certain vowels as combining marks attached to consonant letters. For example, “ह” + “​ि” = “हि” (“h” + “i” = “hi”).

    Korean characters stand for syllables, but they are composed of letters called jamo that stand for the vowels and consonants in the syllable. While there are code points for precomposed Korean syllables, it’s also possible to dynamically compose them by concatenating their jamo. For example, “ᄒ” + “ᅡ” + “ᆫ” = “한” (“h” + “a” + “n” = “han”).

Canonical Equivalence

In Unicode, precomposed characters exist alongside the dynamic composition system. A consequence of this is that there are multiple ways to express “the same” string—different sequences of code points that result in the same user-perceived characters. For example, as we saw earlier, we can express the character “Á” either as the single code point U+00C1, or as the string of two code points U+0041 U+0301.

Another source of ambiguity is the ordering of multiple diacritics in a single character. Diacritic order matters visually when two diacritics apply to the same side of the base character, e.g. both above: “ǡ” (dot, then macron) is different from “ā̇” (macron, then dot). However, when diacritics apply to different sides of the character, e.g. one above and one below, then the order doesn’t affect rendering. Moreover, a character with multiple diacritics might have one of the diacritics precomposed and others expressed as combining marks.

For example, the Vietnamese letter “ệ” can be expressed in five different ways:

    Fully precomposed: U+1EC7 “ệ”
    Partially precomposed: U+1EB9 “ẹ” + U+0302 “◌̂”
    Partially precomposed: U+00EA “ê” + U+0323 “◌̣”
    Fully decomposed: U+0065 “e” + U+0323 “◌̣” + U+0302 “◌̂”
    Fully decomposed: U+0065 “e” + U+0302 “◌̂” + U+0323 “◌̣”

Unicode refers to set of strings like this as “canonically equivalent”. Canonically equivalent strings are supposed to be treated as identical for purposes of searching, sorting, rendering, text selection, and so on. This has implications for how you implement operations on text. For example, if an app has a “find in file” operation and the user searches for “ệ”, it should, by default, find occurrences of any of the five versions of “ệ” above!
Normalization Forms

To address the problem of “how to handle canonically equivalent strings”, Unicode defines several normalization forms: ways of converting strings into a canonical form so that they can be compared code-point-by-code-point (or byte-by-byte).

The “NFD” normalization form fully decomposes every character down to its component base and combining marks, taking apart any precomposed code points in the string. It also sorts the combining marks in each character according to their rendered position, so e.g. diacritics that go below the character come before the ones that go above the character. (It doesn’t reorder diacritics in the same rendered position, since their order matters visually, as previously mentioned.)

The “NFC” form, conversely, puts things back together into precomposed code points as much as possible. If an unusual combination of diacritics is called for, there may not be any precomposed code point for it, in which case NFC still precomposes what it can and leaves any remaining combining marks in place (again ordered by rendered position, as in NFD).

There are also forms called NFKD and NFKC. The “K” here refers to compatibility decompositions, which cover characters that are “similar” in some sense but not visually identical. However, I’m not going to cover that here.
Grapheme Clusters

As we’ve seen, Unicode contains various cases where a thing that a user thinks of as a single “character” might actually be made up of multiple code points under the hood. Unicode formalizes this using the notion of a grapheme cluster: a string of one or more code points that constitute a single “user-perceived character”.

UAX #29 defines the rules for what, precisely, qualifies as a grapheme cluster. It’s approximately “a base code point followed by any number of combining marks”, but the actual definition is a bit more complicated; it accounts for things like Korean jamo, and emoji ZWJ sequences.

The main thing grapheme clusters are used for is text editing: they’re often the most sensible unit for cursor placement and text selection boundaries. Using grapheme clusters for these purposes ensures that you can’t accidentally chop off some diacritics when you copy-and-paste text, that left/right arrow keys always move the cursor by one visible character, and so on.

Another place where grapheme clusters are useful is in enforcing a string length limit—say, on a database field. While the true, underlying limit might be something like the byte length of the string in UTF-8, you wouldn’t want to enforce that by just truncating bytes. At a minimum, you’d want to “round down” to the nearest code point boundary; but even better, round down to the nearest grapheme cluster boundary. Otherwise, you might be corrupting the last character by cutting off a diacritic, or interrupting a jamo sequence or ZWJ sequence.
And More…

There’s much more that could be said about Unicode from a programmer’s perspective! I haven’t gotten into such fun topics as case mapping, collation, compatibility decompositions and confusables, Unicode-aware regexes, or bidirectional text. Nor have I said anything yet about implementation issues—how to efficiently store and look-up data about the sparsely-assigned code points, or how to optimize UTF-8 decoding, string comparison, or NFC normalization. Perhaps I’ll return to some of those things in future posts.

Unicode is a fascinating and complex system. It has a many-to-one mapping between bytes and code points, and on top of that a many-to-one (or, under some circumstances, many-to-many) mapping between code points and “characters”. It has oddball special cases in every corner. But no one ever claimed that representing all written languages was going to be easy, and it’s clear that we’re never going back to the bad old days of a patchwork of incompatible encodings.

Further reading:

    The Unicode Standard
    UTF-8 Everywhere Manifesto
    Dark corners of Unicode by Eevee
    ICU (International Components for Unicode)—C/C++/Java libraries implementing many Unicode algorithms and related things
    Python 3 Unicode Howto
    Google Noto Fonts—set of fonts intended to cover all assigned code points

"""

In [222]:
rawtext

'\nＵｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺\u200c🇳\u200c🇮\u200c🇨\u200c🇴\u200c🇩\u200c🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to “support Unicode” in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don’t blame programmers for still finding the whole thing mysterious, even 30 years after Unicode’s inception.\n\nA few months ago, I got interested in Unicode and decided to spend some time learning more about it in detail. In this article, I’ll give an introduction to it from a programmer’s point of view.\n\nI’m going to focus on the character set and what’s involved in working with strings and files of Unicode text. However, in this article I’m not going to talk about fonts, text layout/shaping/rendering, or localization in detail—thos

In [412]:
# from "A programmers introduction to unicode"
ogtext = rawtext
tokens= ogtext.encode("utf-8") # raw bytes
#tokens = list(tokens)
tokens = list(map(int,tokens)) # convert to a list of integers in range from 0 to 255 -> for convenience
print('---')
print(ogtext)
print('length:', len(ogtext))
print()
print(tokens)
print('length:', len(tokens))
og_tokens = list(tokens)
# NOTE that the length of tokens is longer in some cases because some special characters take up two byte sequences

---

Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺‌🇳‌🇮‌🇨‌🇴‌🇩‌🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to “support Unicode” in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don’t blame programmers for still finding the whole thing mysterious, even 30 years after Unicode’s inception.

A few months ago, I got interested in Unicode and decided to spend some time learning more about it in detail. In this article, I’ll give an introduction to it from a programmer’s point of view.

I’m going to focus on the character set and what’s involved in working with strings and files of Unicode text. However, in this article I’m not going to talk about fonts, text layout/shaping/rendering, or localization in detail—those are separate issues, beyond my

In [264]:
def merge(ids, locs, idx):
    iflag = 0
    tokout = []
    for i in range(len(ids)):
        if iflag > 0:
            iflag -= 1
        else:
            if i in locs:
                iflag=1
                tokout.append(idx)
            else:
                tokout.append(ids[i])
    return tokout

In [410]:
maxfreq=len(tokens)
#maxnum=max(tokens)
maxnum=255
tokenized_values = {}
vocab_size = 276 # limit to just 20 merges

while maxfreq > 1 and maxnum < vocab_size:
    tokcount={}
    tokloc={}
    
    for i in range(len(tokens)-1):
        tokcount[str(tokens[i]) + '_' + str(tokens[i+1])] = tokcount.get(str(tokens[i]) + '_' + str(tokens[i+1]), 0) + 1
        tokloc[str(tokens[i]) + '_' + str(tokens[i+1])] = tokloc.get(str(tokens[i]) + '_' + str(tokens[i+1]), []) + [i] 
    
    maxfreq = max(tokcount.values())
    if maxfreq==1:
        break
        
    maxnum +=1
    tokout=[]
    
    maxloc = tokloc[max(tokcount, key=tokcount.get)]
    new_token = [int(k) for k in max(tokcount, key=tokcount.get).split('_')]
    tokenized_values[maxnum] = new_token
    #print([chr(int(k)) for k in new_token])
    
    iflag=0

    # totally cool pythonic alternative:
    tokout = merge(tokens, maxloc, maxnum)
    
    
    tokens = tokout
        
            
print("max token value used:", max(tokens))
print("max tokens at end:", len(tokens))


max token value used: 276
max tokens at end: 17376


In [270]:
print(''.join([chr(int(k)) for k in tokens]))


ï¼µï½ï½ï½ï½ï½ï½! ð¤ððððððć½ ðºćð³ćð®ćð¨ćð´ćð©ćðª! ð ThĀvĄĐnamĀstrikeĂfeČ ĉċawĀātĔĒheČtĂof programmĄĂwĊldwide. WĀđl know wĀoughĆtĔćsuppĊĆUniďeć ā our softwČĀ(whatevĄ ăaĆmeĉsćlikĀusĎ wchČ_ĆfĊ đl ĒstrĎsĈright?). BuĆUniďĀcĉ bĀabstruseĈĉċdivĎ ātĔĒăousĉd-pagĀUniďĀStĉdČċpluĂitĂdozčĂof supplemčtČĐĉnexesĈrepĊtsĈĉċnoteĂcĉ bĀmĊĀăĉ a littlĀātimidatĎ. I dēćĆblamĀprogrammĄĂfĊ still fādĎ ĒwholĀăĎ mystĄiousĈevč 30 yeČĂaftĄ UniďećĂāceptiē.

A few mēăĂagoĈI goĆātĄesteċā UniďĀĉċdecideċtĔspčċsomĀtimĀleČnĎ mĊĀabouĆiĆā detail. In ăiĂČticleĈIćll givĀĉ ātroductiē tĔiĆfrom a programmĄćĂpoāĆof view.

Ićm goĎ tĔfocuĂē ĒchČactĄ seĆĉċwhatćĂāvolveċā wĊkĎ wiă strĎĂĉċfileĂof UniďĀtext. HowevĄĈā ăiĂČticlĀIćm noĆgoĎ tĔtđk abouĆfētsĈtexĆlayout/shapĎ/rčdĄĎĈĊ locđizatiē ā detailćăosĀČĀsepČatĀissuesĈbeyēċmĐsąpĀ(ĉċknowledge) hĄe.

    DivĄsitĐĉċInhĄčĆComplexity
    ThĀUniďĀCodespace
        CodespacĀAllocatiē
        Scripts
        UsagĀFrequčcy
    EnďĎs
       

In [272]:
tokenized_values

{256: [101, 32],
 257: [105, 110],
 258: [115, 32],
 259: [116, 104],
 260: [101, 114],
 261: [99, 111],
 262: [116, 32],
 263: [226, 128],
 264: [44, 32],
 265: [97, 110],
 266: [111, 114],
 267: [100, 32],
 268: [97, 114],
 269: [101, 110],
 270: [257, 103],
 271: [261, 100],
 272: [121, 32],
 273: [97, 108],
 274: [259, 256],
 275: [111, 110],
 276: [111, 32]}

In [274]:
print("tokens length:", len(og_tokens))
print("ids length:", len(tokens))
print(f"compression ratio: {len(og_tokens)/len(tokens):.2f}X")

tokens length: 24703
ids length: 19453
compression ratio: 1.27X


In [ ]:
# NOTE that the tokenizer is a completely separate module from the LLM - it has its own training set of text (which could be diff from the LLM text)
# and then it translates back and forth between raw text and token sqeuences
# the llm later only ever sees the tokens and never directly deals with any text

In [258]:
# now how do we do the encoding and decoding step:
# note that the amount of diff training text in the tokenizer set informs the density of the token space (via merging)
# ex - if you have a lot of japanese text, you'll get more japanese text merges and shorter sequences, therefore better performance on japanese text

In [414]:
def get_stats(ids):
    counts = {}
    for pair in zip(ids, ids[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    return counts

def merge(ids, pair, idx):
  newids = []
  i = 0
  while i < len(ids):
    if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
      newids.append(idx)
      i += 2
    else:
      newids.append(ids[i])
      i += 1
  return newids

# ---
vocab_size = 276 # the desired final vocabulary size
num_merges = vocab_size - 256
ids = list(tokens) # copy so we don't destroy the original list

merges = {} # (int, int) -> int
for i in range(num_merges):
  stats = get_stats(ids)
  pair = max(stats, key=stats.get)
  idx = 256 + i
  print(f"merging {pair} into a new token {idx}")
  ids = merge(ids, pair, idx)
  merges[pair] = idx

merging (101, 32) into a new token 256
merging (105, 110) into a new token 257
merging (115, 32) into a new token 258
merging (116, 104) into a new token 259
merging (101, 114) into a new token 260
merging (99, 111) into a new token 261
merging (116, 32) into a new token 262
merging (226, 128) into a new token 263
merging (44, 32) into a new token 264
merging (97, 110) into a new token 265
merging (111, 114) into a new token 266
merging (100, 32) into a new token 267
merging (97, 114) into a new token 268
merging (101, 110) into a new token 269
merging (257, 103) into a new token 270
merging (261, 100) into a new token 271
merging (121, 32) into a new token 272
merging (97, 108) into a new token 273
merging (259, 256) into a new token 274
merging (111, 110) into a new token 275


In [400]:
def decode_single_id(id, max_og = 255):
    out = []
    if id <= max_og:
        return [id]
    else:
        new_id = tokenized_values(id)
        flag = 0
        return [decode_single_id(k) for k in tokenized_values[id]]



In [426]:
vocab = {idx: bytes([idx]) for idx in range(256)}
for (p0, p1), idx in merges.items():
    vocab[idx] = vocab[p0] + vocab[p1]



def decode(ids):
    # given ids (list of integers), return python string
    tokens = b"".join(vocab[idx] for idx in ids)
    text = tokens.decode("utf-8", errors='replace')        
    return text

In [432]:
decode(ids)

'\nＵｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺\u200c🇳\u200c🇮\u200c🇨\u200c🇴\u200c🇩\u200c🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to “support Unicode” in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don’t blame programmers for still finding the whole thing mysterious, even 30 years after Unicode’s inception.\n\nA few months ago, I got interested in Unicode and decided to spend some time learning more about it in detail. In this article, I’ll give an introduction to it from a programmer’s point of view.\n\nI’m going to focus on the character set and what’s involved in working with strings and files of Unicode text. However, in this article I’m not going to talk about fonts, text layout/shaping/rendering, or localization in detail—thos

In [444]:
stats = get_stats([1,3,1])
pair = min(stats, key=lambda p: merges.get(p, float("inf")))
pair

(1, 3)

In [440]:
merges

{(101, 32): 256,
 (105, 110): 257,
 (115, 32): 258,
 (116, 104): 259,
 (101, 114): 260,
 (99, 111): 261,
 (116, 32): 262,
 (226, 128): 263,
 (44, 32): 264,
 (97, 110): 265,
 (111, 114): 266,
 (100, 32): 267,
 (97, 114): 268,
 (101, 110): 269,
 (257, 103): 270,
 (261, 100): 271,
 (121, 32): 272,
 (97, 108): 273,
 (259, 256): 274,
 (111, 110): 275}

In [442]:
get_stats([1,3,1])

{(1, 3): 1, (3, 1): 1}

In [459]:
def encode(text):
    # given a string, return a list of integers (the tokens)
    tokens = list(text.encode("utf-8")) # starting token
    # merge now
    if len(text) >= 2:
        while True:
            stats = get_stats(tokens)
            pair = min(stats, key=lambda p: merges.get(p, float("inf"))) # float inf makes sure any non-merge pair is coded as infinity
            if pair not in merges:
                break # once we don't find a merge candidate we break
    
            idx = merges[pair]
            tokens = merge(tokens, pair, idx)
    
    return tokens



print(encode("h"))

[104]


In [463]:
# not all decode/encode operations return exactly the same input -> if there are invalid utf-8 characters, we can't properly identify them
# BUT the training text should always be decode then encode-able
text2 = decode(encode(ogtext))
print(text2 == ogtext)

True


In [469]:
valtext = """ Unicode, formally The Unicode Standard,[note 1] is a text encoding standard maintained by the Unicode Consortium designed to support the use of text in all of the world's writing systems that can be digitized. Version 16.0 of the standard[A] defines 154998 characters and 168 scripts[3] used in various ordinary, literary, academic, and technical contexts.

Many common characters, including numerals, punctuation, and other symbols, are unified within the standard and are not treated as specific to any given writing system. Unicode encodes 3790 emoji, with the continued development thereof conducted by the Consortium as a part of the standard.[4] Moreover, the widespread adoption of Unicode was in large part responsible for the initial popularization of emoji outside of Japan. Unicode is ultimately capable of encoding more than 1.1 million characters.

Unicode has largely supplanted the previous environment of a myriad of incompatible character sets, each used within different locales and on different computer architectures. Unicode is used to encode the vast majority of text on the Internet, including most web pages, and relevant Unicode support has become a common consideration in contemporary software development.

The Unicode character repertoire is synchronized with ISO/IEC 10646, each being code-for-code identical with one another. However, The Unicode Standard is more than just a repertoire within which characters are assigned. To aid developers and designers, the standard also provides charts and reference data, as well as annexes explaining concepts germane to various scripts, providing guidance for their implementation. Topics covered by these annexes include character normalization, character composition and decomposition, collation, and directionality.[5]

Unicode text is processed and stored as binary data using one of several encodings, which define how to translate the standard's abstracted codes for characters into sequences of bytes. The Unicode Standard itself defines three encodings: UTF-8, UTF-16, and UTF-32, though several others exist. Of these, UTF-8 is the most widely used by a large margin, in part due to its backwards-compatibility with ASCII.
Origin and development

Unicode was originally designed with the intent of transcending limitations present in all text encodings designed up to that point: each encoding was relied upon for use in its own context, but with no particular expectation of compatibility with any other. Indeed, any two encodings chosen were often totally unworkable when used together, with text encoded in one interpreted as garbage characters by the other. Most encodings had only been designed to facilitate interoperation between a handful of scripts—often primarily between a given script and Latin characters—not between a large number of scripts, and not with all of the scripts supported being treated in a consistent manner.

The philosophy that underpins Unicode seeks to encode the underlying characters—graphemes and grapheme-like units—rather than graphical distinctions considered mere variant glyphs thereof, that are instead best handled by the typeface, through the use of markup, or by some other means. In particularly complex cases, such as the treatment of orthographical variants in Han characters, there is considerable disagreement regarding which differences justify their own encodings, and which are only graphical variants of other characters.

At the most abstract level, Unicode assigns a unique number called a code point to each character. Many issues of visual representation—including size, shape, and style—are intended to be up to the discretion of the software actually rendering the text, such as a web browser or word processor. However, partially with the intent of encouraging rapid adoption, the simplicity of this original model has become somewhat more elaborate over time, and various pragmatic concessions have been made over the course of the standard's development.

The first 256 code points mirror the ISO/IEC 8859-1 standard, with the intent of trivializing the conversion of text already written in Western European scripts. To preserve the distinctions made by different legacy encodings, therefore allowing for conversion between them and Unicode without any loss of information, many characters nearly identical to others, in both appearance and intended function, were given distinct code points. For example, the Halfwidth and Fullwidth Forms block encompasses a full semantic duplicate of the Latin alphabet, because legacy CJK encodings contained both "fullwidth" (matching the width of CJK characters) and "halfwidth" (matching ordinary Latin script) characters.

The Unicode Bulldog Award is given to people deemed to be influential in Unicode's development, with recipients including Tatsuo Kobayashi, Thomas Milo, Roozbeh Pournader, Ken Lunde, and Michael Everson.[6] 
"""
valtext2 = decode(encode(valtext))
print(valtext2 == valtext)
# generally, we should be able to see that validation text (other random text we grab) will work properly

True


In [402]:
decode_single_id(274)

[[116, 104], [101, 32]]

In [473]:
# one important note (from gpt2 encoding paper) -> common words like dog
# can be dog. dog! dog? -> and this can eat up a lot of variable space
# so we want to MANUALLY enforce that some types of characters should never be merged together
# relevant code: https://github.com/openai/gpt-2

In [ ]:
# forced splits using regex patterns (gpt series)

In [491]:
import regex as re
gpt2pat = re.compile(r"""'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+""")

print(re.findall(gpt2pat, "Hello've world123, how's HOW'S are. are       you!!!?"))
# NOTE: separates out punctuation, non-letters, and specific appendices, and tokenizer will then process the list piece by piece and concatenate back 
# this enforces that some merges will NOT happen
# note - not ideal that gpt2 has explicitly separated out apostraphes because it lacks some flexibility in unicode statements
# should also IGNORE CASE beacause upper/lower case will separate apostraphe differently
# ADDITIONAL NOTE: convention is to have the whitespace (single space) prior to a word; seems to work well
# also keeps whitespace chunks together

['Hello', "'ve", ' world', '123', ',', ' how', "'s", ' HOW', "'", 'S', ' are', '.', ' are', '      ', ' you', '!!!?']


In [497]:
# additional example through code
example = """
for i in range(1, 101):
    if i % 3 == 0 and i % 5 == 0:
        print("FizzBuzz")
    elif i % 3 == 0:
        print("Fizz")
    elif i % 5 == 0:
        print("Buzz")
"""
print(re.findall(gpt2pat, example))
# note that for GPT2 - spaces were always just a single space and never were concatenated in the tokenizer

['\n', 'for', ' i', ' in', ' range', '(', '1', ',', ' 101', '):', '\n   ', ' if', ' i', ' %', ' 3', ' ==', ' 0', ' and', ' i', ' %', ' 5', ' ==', ' 0', ':', '\n       ', ' print', '("', 'FizzBuzz', '")', '\n   ', ' elif', ' i', ' %', ' 3', ' ==', ' 0', ':', '\n       ', ' print', '("', 'Fizz', '")', '\n   ', ' elif', ' i', ' %', ' 5', ' ==', ' 0', ':', '\n       ', ' print', '("', 'Buzz', '")', '\n']


In [501]:
%pip install tiktoken

   ---------------------------------------- 0.0/799.3 kB ? eta -:--:--
   - -------------------------------------- 30.7/799.3 kB 1.4 MB/s eta 0:00:01
   --------------------------------------  798.7/799.3 kB 12.7 MB/s eta 0:00:01
   --------------------------------------- 799.3/799.3 kB 10.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [502]:
# tiktoken library for openAI - official library for tokenizer!
import tiktoken

# GPT-2 (does not merge spaces)
enc = tiktoken.get_encoding("gpt2")
print(enc.encode("     hello world!!!"))

# GPT-4 (does not merge spaces)
enc = tiktoken.get_encoding("cl100k_base")
print(enc.encode("     hello world!!!"))

# we can look through the gpt docs for tiktoken_ext to see how regexes work in gpt2 and gpt4
# some additional whitespace handling
# no case sensitivity
# they will never merge numbers more than 3 digits long 



[220, 220, 220, 220, 23748, 995, 10185]
[257, 24748, 1917, 12340]


In [505]:
# NEXT SECTION: going through the encoder.py file for gpt-2 repo (publicly avail) - need to get the windows version
!wget https://openaipublic.blob.core.windows.net/gpt-2/models/1558M/vocab.bpe
!wget https://openaipublic.blob.core.windows.net/gpt-2/models/1558M/encoder.json
# their "encoder" is exactly equivalent to their vocab

'wget' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.


In [509]:
import os, json

with open('encoder.json', 'r') as f:
    encoder = json.load(f)

with open('vocab.bpe', 'r', encoding="utf-8") as f:
    bpe_data = f.read()
bpe_merges = [tuple(merge_str.split()) for merge_str in bpe_data.split('\n')[1:-1]]

In [ ]:
# interesting note: there is a seperate layer used with the tokenization ->  byte encode, encode; decode, byte decode (interesting but not useful)
# otherwise the encoder/decoder are very similar to what we've done above


In [523]:
# SPECIAL TOKENS!
len(encoder) # why 50257? -> 256 raw byte tokens, 50,000 merges, PLUS an end of text token
# Note that special token used pervasively in language modeling, fine tuning stage to delimit conversations between bot and end user
# so we have other tokens -> <|im_start|>, <|im_end|> etc
# you can FORK the tiktoken repo to check out tokens & modify your own
# FIM prefix, middle, suffix -> special tokens to "fill in middle"
# endofprompt, endoftext also used as special tokens
# NOTE that you ahve to modify the neural network any time you change the tokens -> embedding matrix for vocab and output layer need to be changed!
# now we have enough to build a gpt-4-like tokenizer!


50257

In [3]:
def get_stats(ids):
    counts = {}
    for pair in zip(ids, ids[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    return counts

def merge(ids, pair, idx):
  newids = []
  i = 0
  while i < len(ids):
    if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
      newids.append(idx)
      i += 2
    else:
      newids.append(ids[i])
      i += 1
  return newids


In [107]:
class BasicTokenizer:

    def __init__(self):
        pass

    
    def train(self, text, vocab_size, verbose=False):
        # create tokens
        tokens= text.encode("utf-8") # raw bytes
        tokens = list(map(int,tokens)) # convert to a list of integers in range from 0 to 255 -> for convenience

        # set up merges and vocab size
        num_merges = vocab_size - 256
        self.ids = list(tokens) # initializing token list
        self.merges = {} # (int, int) -> int

        # loop to create new tokens via byte-pair encoding
        for i in range(num_merges):
          stats = get_stats(self.ids)
          pair = max(stats, key=stats.get)
          idx = 256 + i
          if verbose:
              print(f"merging {pair} into a new token {idx}")
          self.ids = merge(self.ids, pair, idx)
          self.merges[pair] = idx
          progressbar(i, num_merges-1, "Byte-Pair Encoding Progress: ")
        print()
        self.vocab = {idx: bytes([idx]) for idx in range(256)}
        for (p0, p1), idx in self.merges.items():
            self.vocab[idx] = self.vocab[p0] + self.vocab[p1]

        return self.merges, self.ids


    def encode(self, text):
        # given a string, return a list of integers (the tokens)
        tokens = list(text.encode("utf-8")) # starting token
        # merge now
        if len(text) >= 2:
            while True:
                stats = get_stats(tokens)
                pair = min(stats, key=lambda p: self.merges.get(p, float("inf"))) # float inf makes sure any non-merge pair is coded as infinity
                if pair not in self.merges:
                    break # once we don't find a merge candidate we break
        
                idx = self.merges[pair]
                tokens = merge(tokens, pair, idx)
        
        return tokens



    def decode(self, ids):
        # given ids (list of integers), return python string
        tokens = b"".join(self.vocab[idx] for idx in ids)
        text = tokens.decode("utf-8", errors='replace')        
        return text




In [103]:
with open('taylorswift.txt', 'r', encoding="utf-8") as f:
    text = f.read()

In [109]:
tokenboi=BasicTokenizer()
_, _ = tokenboi.train(text, 1000)
tokend = tokenboi.encode("hello world")
tokenboi.decode(tokend)

Byte-Pair Encoding Progress: |==================================================|


'hello world'

# Adding Regex splits to the mix

In [93]:
def progressbar(it, maxval, front_text=''):
    prog=int(it/maxval*50)
    #print(prog)
    a=''.join(["="]*prog)
    b=''.join([" "]*(50-prog))
    print(front_text + "|"+a+b+'|', end='\r')

In [111]:
import os, sys, time
import regex as re

In [113]:
GPT4_SPLIT_PATTERN = r"""'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?+\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]++[\r\n]*|\s*[\r\n]|\s+(?!\S)|\s+"""

In [203]:
GPT4_SPECIAL_TOKENS = {
    '<|endoftext|>': 100257,
    '<|fim_prefix|>': 100258,
    '<|fim_middle|>': 100259,
    '<|fim_suffix|>': 100260,
    '<|endofprompt|>': 100276
}

In [169]:
def get_stats_list(ids_list):
    counts = {}
    for ids in ids_list:
        for pair in zip(ids, ids[1:]):
            counts[pair] = counts.get(pair, 0) + 1
    return counts

def merge(ids, pair, idx):
  newids = []
  i = 0
  while i < len(ids):
    if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
      newids.append(idx)
      i += 2
    else:
      newids.append(ids[i])
      i += 1
  return newids

In [ ]:
def recover_merges(mergeable_ranks):
    # the `merges` are already the byte sequences in their merged state.
    # so we have to recover the original pairings. We can do this by doing
    # a small BPE training run on all the tokens, in their order.
    # also see https://github.com/openai/tiktoken/issues/60
    # also see https://github.com/karpathy/minbpe/issues/11#issuecomment-1950805306
    merges = {}
    for token, rank in mergeable_ranks.items():
        if len(token) == 1:
            continue # skip raw bytes
        pair = tuple(bpe(mergeable_ranks, token, max_rank=rank))
        assert len(pair) == 2
        # recover the integer ranks of the pair
        ix0 = mergeable_ranks[pair[0]]
        ix1 = mergeable_ranks[pair[1]]
        merges[(ix0, ix1)] = rank

    return merges

In [193]:
class RegexTokenizer:

    def __init__(self):
        pass


    def get_stats_list(self, ids_list):
        counts = {}
        for ids in ids_list:
            for pair in zip(ids, ids[1:]):
                counts[pair] = counts.get(pair, 0) + 1
        return counts 

    
        
    
    def train(self, text, vocab_size, regex_pattern, verbose=False):
        # create tokens

        patcomp = re.compile(regex_pattern)
        sptext = re.findall(patcomp,text)
        
        tokens= [t.encode("utf-8") for t in sptext] # raw bytes
        tokens = [list(map(int,t)) for t in tokens] # convert to a list of integers in range from 0 to 255 -> for convenience

        # set up merges and vocab size
        num_merges = vocab_size - 256
        self.ids = [list(t) for t in tokens] # initializing token list (list of lists)
        self.merges = {} # (int, int) -> int

        # loop to create new tokens via byte-pair encoding
        for i in range(num_merges):
          
            stats = self.get_stats_list(self.ids)

            if len(stats) == 0 or max([len(id) for id in self.ids]) == 1:
                print()
                print('reached the end of mergeable tokens for training text')
                print('total tokens: ' + str(idx))
                break
            pair = max(stats, key=stats.get)
            idx = 256 + i
            if verbose:
                print(f"merging {pair} into a new token {idx}")
            self.ids = [merge(id, pair, idx) for id in self.ids]
            self.merges[pair] = idx
            progressbar(i, num_merges-1, "Byte-Pair Encoding Progress: ")
        print()
        self.vocab = {idx: bytes([idx]) for idx in range(256)}
        print('Token Merging Complete.')
        for (p0, p1), idx in self.merges.items():
            self.vocab[idx] = self.vocab[p0] + self.vocab[p1]

        return self.merges, self.ids


    def encode(self, text):
        # given a string, return a list of integers (the tokens)
        tokens = list(text.encode("utf-8")) # starting token
        # merge now
        if len(text) >= 2:
            while True:
                stats = get_stats(tokens)
                pair = min(stats, key=lambda p: self.merges.get(p, float("inf"))) # float inf makes sure any non-merge pair is coded as infinity
                if pair not in self.merges:
                    break # once we don't find a merge candidate we break
        
                idx = self.merges[pair]
                tokens = merge(tokens, pair, idx)
        
        return tokens



    def decode(self, ids):
        # given ids (list of integers), return python string
        tokens = b"".join(self.vocab[idx] for idx in ids)
        text = tokens.decode("utf-8", errors='replace')        
        return text




In [195]:
tokenbb=RegexTokenizer()
_, _ = tokenbb.train(text, 100000, GPT4_SPLIT_PATTERN)
tokendo = tokenbb.encode("hello world!!!? (안녕하세요!) lol123 😉")
tokenbb.decode(tokendo)

Byte-Pair Encoding Progress: |=====                                             |
reached the end of mergeable tokens for training text
total tokens: 10416

Token Merging Complete.


'hello world!!!? (안녕하세요!) lol123 😉'

In [199]:
import tiktoken
import tiktoken
enc = tiktoken.get_encoding("cl100k_base") # this is the GPT-4 tokenizer
ids = enc.encode("hello world!!!? (안녕하세요!) lol123 😉")
text = enc.decode(ids) # get the same text back

In [214]:
print(enc.encode("hello world!!!? (안녕하세요!) lol123 😉"))
print(enc.decode(enc.encode("hello world!!!? (안녕하세요!) lol123 😉")) == "hello world!!!? (안녕하😉요!) lol123 😉")

[15339, 1917, 12340, 30, 320, 31495, 230, 75265, 243, 92245, 16715, 28509, 4513, 57037]
True


In [183]:
idlist=[]
for i in tokenbb.ids:
    for j in i:
        idlist.append(j)
        

In [191]:
len(get_stats_list(tokenbb.ids))

0

In [216]:

def recover_merges(mergeable_ranks):
    # the `merges` are already the byte sequences in their merged state.
    # so we have to recover the original pairings. We can do this by doing
    # a small BPE training run on all the tokens, in their order.
    # also see https://github.com/openai/tiktoken/issues/60
    # also see https://github.com/karpathy/minbpe/issues/11#issuecomment-1950805306
    merges = {}
    for token, rank in mergeable_ranks.items():
        if len(token) == 1:
            continue # skip raw bytes
        pair = tuple(bpe(mergeable_ranks, token, max_rank=rank))
        assert len(pair) == 2
        # recover the integer ranks of the pair
        ix0 = mergeable_ranks[pair[0]]
        ix1 = mergeable_ranks[pair[1]]
        merges[(ix0, ix1)] = rank

    return merges


def bpe(mergeable_ranks, token, max_rank):
    # helper function used in get_gpt4_merges() to reconstruct the merge forest
    parts = [bytes([b]) for b in token]
    while True:
        min_idx = None
        min_rank = None
        for i, pair in enumerate(zip(parts[:-1], parts[1:])):
            rank = mergeable_ranks.get(pair[0] + pair[1])
            if rank is not None and (min_rank is None or rank < min_rank):
                min_idx = i
                min_rank = rank
        if min_rank is None or (max_rank is not None and min_rank >= max_rank):
            break
        assert min_idx is not None
        parts = parts[:min_idx] + [parts[min_idx] + parts[min_idx + 1]] + parts[min_idx + 2:]
    return parts

In [234]:
[bytes(b, encoding='utf-8') for b in 'hello this is dog!😉😉']

[b'h',
 b'e',
 b'l',
 b'l',
 b'o',
 b' ',
 b't',
 b'h',
 b'i',
 b's',
 b' ',
 b'i',
 b's',
 b' ',
 b'd',
 b'o',
 b'g',
 b'!',
 b'\xf0\x9f\x98\x89',
 b'\xf0\x9f\x98\x89']

# Sentencepiece - commonly used bc it can efficiently train and inference BPE tokenizers

In [1]:
# Sentencepiece is a little different
# with tiktoken -> we encode to utf-8 and merge using bytes
# with sentencepiece -> it looks directly at the codepoints themselves
    # if there are codepoints that don't come up so often (super rare) -> might get an unknown 
    # OR if you have the (byte_fallback) option turned on, it'll encode to utf-8, translate to new tokens
# AK sees tiktoken to be much cleaner overall, though


In [2]:
%pip install sentencepiece

     -------------------------------------- 991.5/991.5 kB 4.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [6]:
import sentencepiece as spm
import os

In [5]:
# write a toy.txt file with some random text
with open('toy.txt', 'w', encoding='utf-8') as f:
    f.write("sentencepiece is an unupervised text tokenizer and decoder mainly used for Neural Networ-based text generation. They said I can write whatever I want so let's stream of consciousness this mfer. It's gorgeous out right now and that's why I'm writing this from the patio - but I have no idea why I would decide to write about the weather. I guess it's just because most people talk about it so it becomes just, like, a default way of talking about things. It is really special here, though. The view is unbeatable and I think this is the kind of place that I would be cool retiring to. Maybe somewere with a little more accessibility but overall it's quiet and peaceful and the bridge is a great view. I wonder what the old version looked like - I heard it was kinda crappy and loud but who knows. It probably won't be worse than this. I can only wonder, though, if anything happens with this area. We are fairly close to the waterline and if any flooding occurs, this place is close to underwater. I think we have a good 10-15 feet and we're not in a low spot per se, but this place could get a lot less accessible and erosion could be a potential issue if things deteriorate in 20 or so years. Hopefully by the time I have to retire, this place will still be in nice condition and we can enjoy the place once again. Okay back to work.")

Docs for sentence piece options:
- markdown
- protobuf 

(there were links here I can't access)

In [18]:
# sentencepiece has been around for a while and accounts for a lot of history (good and bad things)
# train a sentencepiece model on toy.txt
# the settings here are (best effort) those used for training llama 2
import os

options = dict(
    # input spec
    input='toy.txt',
    input_format='text',
    
    # output spec
    model_prefix='tok400', #output filename prefix
    
    #algorithm spec
    #bpe alg
    model_type='bpe',
    vocab_size=400,
    
    #normalization - used to be prevalent in NLP BEFORE LLMs
    normalization_rule_name='identity', #turn off normalization
    remove_extra_whitespaces=False,
    input_sentence_size=2000000000, #max number of training sentences
    max_sentence_length=4192, # max number of bytes per sentence
    seed_sentencepiece_size=1000000,
    shuffle_input_sentence=True,
    
    # rare word treatment
    character_coverage=0.99995, # if a character is (1 - THIS) rare, it's excluded from our vocab
    byte_fallback=True,
    
    # merge rules
    split_digits=True,
    split_by_unicode_script=True,
    split_by_whitespace=True,
    split_by_number=True,
    max_sentencepiece_length=16,
    add_dummy_prefix=True,
    allow_whitespace_only_pieces=True,
    
    #special tokens
    unk_id=0, #the unk token MUST exist
    bos_id=1, # the others are optional - set to -1 to turn off
    eos_id=2, #end of sentence
    pad_id=-1, 
    
    #systems
    num_threads=os.cpu_count()
    
)
spm.SentencePieceTrainer.train(**options)

In [19]:
sp = spm.SentencePieceProcessor()
sp.load('tok400.model')
vocab= [[sp.id_to_piece(idx), idx] for idx in range(sp.get_piece_size())]

In [11]:
vocab

[['<unk>', 0],
 ['<s>', 1],
 ['</s>', 2],
 ['<0x00>', 3],
 ['<0x01>', 4],
 ['<0x02>', 5],
 ['<0x03>', 6],
 ['<0x04>', 7],
 ['<0x05>', 8],
 ['<0x06>', 9],
 ['<0x07>', 10],
 ['<0x08>', 11],
 ['<0x09>', 12],
 ['<0x0A>', 13],
 ['<0x0B>', 14],
 ['<0x0C>', 15],
 ['<0x0D>', 16],
 ['<0x0E>', 17],
 ['<0x0F>', 18],
 ['<0x10>', 19],
 ['<0x11>', 20],
 ['<0x12>', 21],
 ['<0x13>', 22],
 ['<0x14>', 23],
 ['<0x15>', 24],
 ['<0x16>', 25],
 ['<0x17>', 26],
 ['<0x18>', 27],
 ['<0x19>', 28],
 ['<0x1A>', 29],
 ['<0x1B>', 30],
 ['<0x1C>', 31],
 ['<0x1D>', 32],
 ['<0x1E>', 33],
 ['<0x1F>', 34],
 ['<0x20>', 35],
 ['<0x21>', 36],
 ['<0x22>', 37],
 ['<0x23>', 38],
 ['<0x24>', 39],
 ['<0x25>', 40],
 ['<0x26>', 41],
 ['<0x27>', 42],
 ['<0x28>', 43],
 ['<0x29>', 44],
 ['<0x2A>', 45],
 ['<0x2B>', 46],
 ['<0x2C>', 47],
 ['<0x2D>', 48],
 ['<0x2E>', 49],
 ['<0x2F>', 50],
 ['<0x30>', 51],
 ['<0x31>', 52],
 ['<0x32>', 53],
 ['<0x33>', 54],
 ['<0x34>', 55],
 ['<0x35>', 56],
 ['<0x36>', 57],
 ['<0x37>', 58],
 ['<0x38>', 5

In [13]:
# note that the korean characters and emoji were not part of the trianing set - they still return values
# we had byte_fallback so it "fell back" to bytes when it encountered unknown tokens
ids = sp.encode("hello world!!!? (안녕하세요!) lol123 😉")
print(ids)

[293, 359, 366, 366, 363, 261, 299, 298, 36, 36, 36, 66, 358, 43, 239, 152, 139, 238, 136, 152, 240, 152, 155, 239, 135, 187, 239, 157, 151, 36, 44, 278, 363, 366, 388, 392, 54, 358, 243, 162, 155, 140]


In [14]:
print([sp.id_to_piece(idx) for idx in ids])

['▁h', 'e', 'l', 'l', 'o', '▁w', 'or', 'ld', '<0x21>', '<0x21>', '<0x21>', '<0x3F>', '▁', '<0x28>', '<0xEC>', '<0x95>', '<0x88>', '<0xEB>', '<0x85>', '<0x95>', '<0xED>', '<0x95>', '<0x98>', '<0xEC>', '<0x84>', '<0xB8>', '<0xEC>', '<0x9A>', '<0x94>', '<0x21>', '<0x29>', '▁l', 'o', 'l', '1', '2', '<0x33>', '▁', '<0xF0>', '<0x9F>', '<0x98>', '<0x89>']


In [17]:
# This is a run with byte_fallback=False
ids = sp.encode("hello world!!!? (안녕하세요!) lol123 😉")
print(ids)
print([sp.id_to_piece(idx) for idx in ids])

[37, 359, 366, 102, 146, 42, 0, 358, 0, 22, 363, 366, 388, 392, 0, 358, 0]
['▁h', 'e', 'l', 'lo', '▁wor', 'ld', '<unk>', '▁', '<unk>', '▁l', 'o', 'l', '1', '2', '<unk>', '▁', '<unk>']


In [20]:
# interesting side-note -> we have a STARTING whitespace
# this is because of add_dummy_prefix -> tries to treat "_[word]" the same as just "[word]"

In [21]:
# summary here: there's a lot of baggage in sentencepiece so it's not the most ideal
# but it is pretty widely used because it can encode and perform inference all in one 
# and it's pretty easy to set up and use to train a tokenizer
# (though tiktoken may be a little better)

In [22]:
# what if we want to take a pre-trained model and EXTEND it? 
# e.g. if you're adding new tokens for new chats
# -> we would need to add rows to the token_embedding_table
    # then initialize the values to be small random nubmers
    # then extend the weight inside the nn.Linear layer -> dot products for new probability
    # it's a very MILD model surgery and it can be done pretty easily
    # only training new parameters to train the model's new tokens, not a full-in retrain


In [ ]:
# MORE functionality -> PAPER: learning to compress prompts with "gist tokens"
# suppose you need a LONG prompt -> it slows everything down to encode a large prompt
# instead, there's a new token (a few new for a sequence), then trian model by distillation
# train representation of new embeddings then optimize over new tokens such that the behavior of new model is identical
# to model with very long prompt
# COMPRESSING to just a few tokens to stand in for very long prompt with almost identical prmpt


In [23]:
# other note: transformers for OTHER input modalities (video, audio, etc)
# how to feed in or predict??
# NO -> stick with transformer, then tokeninze input domains and pretend it's text tokens
# early paper: taming transformers for high resolution imag synthesis (a.k.a #VQGAN)
# other one -> openAI SORA -> turning visual data into patches; chunkate vieos into tokens (hard- autoregressive or soft - diffusion) models


In [ ]:
# SUMMARY!
# why can't LLMs spell -> tokenization! - in some cases there might be TOO MANY tokens and some large words are a single token
    # in some cases you can trick an LLM to spelling right by telling it to split out a sentence into chars separated by spaces
# why are LLMs bad with translation?
    # low training on other language data, and often more bloated and diffuse tokens for other languages as a result
# why are LLMs bad at simple arithmetic?
    # tokenizing some numbers! -> usually aritmetic is done by single digits and tokenization merges often
    # side note - "integer tokenization is insane" explores how numbers are tokenized and it's a weird grouping
# why is gpt-2 have trouble with python?
    # partly the model, but also tokenization did not handle spaces well. Now, tokenization is trained much more on python
# "my llm halts when you print a string <|endoftext|>"?
    # it might parse this as a token with weird token handling logic and REMOVES these special characters
    # similar to sanitizing inputs for sql safety "attack surface"
# weird error about "trailing whitespace"?
    # adding a space at the end of a prompt gives a warning about performance
    # space character is always (or almost always) a prefix to a word or part of word from BPE
    # since the model has seen VERY LITTLE DATA with just a space, now we're out of distribution and it does weird things
    # fundamental issue: LLM is operating on tokens which are TEXT CHUNKS, not characters
    # interesting example -> prompt of .DefaultCellSty (first part of .DefaultCellStyle) predicts an end of string token
    # UNSTABLE TOKENS
# SolidGoldMagikarp? ("SolidGoldMagikarp (plus, prompt generation)")
    # internet famous! guy went to the embedding stable and saw a cluster of tokens that look superstrange inc SGM
    # if you ask the model about these tokens it HALLUCINATES
    # there are some "trigger words" that freak the model out and gives it super nonstandard behavior
    # SolidGoldMagikarp is probably a prolific reddit user and the training set included a lot of reddit data
    # BUT the token is not in the training data so it is essentially "UNALLOCATED MEMORY" and creates undefined behavior
# different formats/languages are more or less efficient with tokenizers
    # json is super dense in tokens, yaml is very efficient 
    
